# Neural networks using tensorflow

## Import libraries

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

import matplotlib.pyplot as plt
import netCDF4 as n
%matplotlib inline
from utils import train_test_split
from sklearn import tree
from sklearn import metrics
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor

## Import data

In [5]:
path = "./files/"
filenames = ["specific_humidity_Europa_sp.nc", "relative_humidity_Europa_sp.nc", "pressure_Europa_sp.nc",  
             "temperature_Europa_sp.nc", "total_cloud_cover_Europa_sp.nc"]


cloud = n.Dataset(path + filenames[-1], "r")
relative = n.Dataset(path + filenames[1], "r")
specific = n.Dataset(path + filenames[0], "r")
pressure = n.Dataset(path + filenames[2], "r")
temperature = n.Dataset(path + filenames[3], "r")

my_files = {'cloud':cloud, 'relative':relative, 'specific':specific, 'pressure':pressure, 'temperature':temperature}


tcc = cloud.variables["tcc"]
rh = relative.variables["r"]
q = specific.variables["q"]
sp = pressure.variables["sp"]
t2m = temperature.variables["t2m"]

my_vars = {'tcc':tcc, 'rh':rh, 'q':q, 'sp':sp, 't2m':t2m}


X = np.empty((tcc.shape[1]*tcc.shape[2], 4))
y = np.empty((tcc.shape[1]*tcc.shape[2], 1))

i=0
for key, value in my_vars.items():
    if key == 'tcc':
        y = np.array(value[0, : , :]).flatten()
    elif key == 'rh' or key == 'q':
        X[:, i] = np.array(value[0, 0, : , :]).flatten()
        i = i+1
    else:
        X[:, i] = np.array(value[0, : , :]).flatten()
        i=i+1

### Normalize and split

In [7]:
Xn = (X - X.mean())/X.std()
yn = (y - y.mean())/y.std()

Xn_train, Xn_test, yn_train, yn_test = train_test_split(Xn, yn, split_size=0.3)

In [14]:
Xn_train = tf.cast(Xn_train, tf.float32)  # convert to float 32 so that we can use tf.matmul

## Neural net model

Feed forward neural net with 2 layers. Activation function: Sigmoid for all layers.

TODO: write as function

### Define weights and biases

In [10]:
npred = Xn_train.shape[1]  # number of predictors

In [11]:
# Using normal distribution (maybe change this)

W_1 = tf.Variable(tf.random_uniform([npred,10]))
b_1 = tf.Variable(tf.zeros([10]))

### Define layer 1 and activation f

In [17]:
layer_1 = tf.add(tf.matmul(Xn_train,W_1), b_1)
# layer_1 = tf.nn.relu(layer_1)
layer_1 = tf.nn.sigmoid(layer_1)  

In [18]:
layer_1 = tf.nn.relu(layer_1)

### Multiply by layer 1 and add bias. Then activation function
#### Obtain layer 2

In [22]:
W_2 = tf.Variable(tf.random_uniform([10,10]))
b_2 = tf.Variable(tf.zeros([10]))
layer_2 = tf.add(tf.matmul(layer_1,W_2), b_2)
layer_2 = tf.nn.sigmoid(layer_2)

### Multiply by layer 2 and add bias. Then activation function
#### Obtain output

In [21]:
W_O = tf.Variable(tf.random_uniform([10,1]))
b_O = tf.Variable(tf.zeros([1]))
output = tf.add(tf.matmul(layer_2,W_O), b_O)

### MSE

In [24]:
#xs = tf.placeholder("float")
#ys = tf.placeholder("float")
#output = neural_net_model(xs,3)
cost = tf.reduce_mean(tf.square(output-yn_train))

### Gradient descent

In [26]:
eta = 0.001
train = tf.train.GradientDescentOptimizer(eta).minimize(cost)